# Thermal Emission

The first example we'll look at is that of thermal emission from a galaxy cluster. In this case, the gas in the core of the cluster is "sloshing" in the center, producing spiral-shaped cold fronts. The dataset we want to use for this example is available for download from the [yt Project](http://yt-project.org) at [this link](http://yt-project.org/data/GasSloshing.tar.gz). 

First, import our necessary modules:

In [ ]:
import yt
import pyxsim
import soxs

Next, we `load` the dataset with yt. Note that this dataset does not have 
species fields in it, so we'll set `default_species_fields="ionized"` to 
assume full ionization (as appropriate for galaxy clusters):

In [ ]:
ds = yt.load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150", 
             default_species_fields="ionized")

Let's use yt to take a slice of density and temperature through the center of the dataset so we can see what we're looking at: 

In [ ]:
slc = yt.SlicePlot(ds, "z", [("gas","density"), ("gas","temperature")], width=(1.0,"Mpc"))
slc.show()

Ok, sloshing gas as advertised. Next, we'll create a sphere object to serve as a source for our investigations. Place it at the center of the domain with `"c"`, and use a radius of 500 kpc:

In [ ]:
sp = ds.sphere("c", (500.,"kpc"))

Now, we need to set up a source model. We said we were going to look at the thermal emission from the hot plasma, which in a galaxy cluster is in collisional ionization equilibrium (CIE), so to do that we can set up a `CIESourceModel`. The first argument specifies which model we want to use. Let's use `"spex"`. The next three arguments are the maximum and minimum energies, and the number of bins in the spectrum. We've chosen these numbers so that the spectrum has an energy resolution of about 1 eV. 

`CIESourceModel` takes a lot of optional arguments, which you can investigate in the docs, but here we'll do something simple and say that the metallicity is a constant $Z = 0.3~Z_\odot$:

In [ ]:
source_model = pyxsim.CIESourceModel("spex", 0.05, 11.0, 1000, 0.3, binscale='log')

We can use this `source_model` object to compute X-ray fields for use in yt calculations. For example, we can create fields for emissivity, luminosity, and photon emissivity within a particular band, in this case 0.5-7 keV:

In [ ]:
xray_fields = source_model.make_source_fields(ds, 0.5, 7.0)
print(xray_fields)

From this we can compute the total luminosity in our sphere object:

In [ ]:
print(sp["gas","xray_luminosity_0.5_7.0_keV"])
print(sp.sum(("gas","xray_luminosity_0.5_7.0_keV")))

and we can make a projection of the emissivity:

In [ ]:
prj = yt.ProjectionPlot(ds, "z", ("gas", "xray_emissivity_0.5_7.0_keV"), width=(1.0,"Mpc"))
prj.show()


We're almost ready to go to generate the photons from this source, but first we should decide what our redshift, collecting area, and exposure time should be. Let's pick big numbers, because remember the point of this first step is to create a Monte-Carlo sample from which to draw smaller sub-samples for mock observations. Note these are all (value, unit) tuples:

In [ ]:
exp_time = (300., "ks") # exposure time
area = (1000.0, "cm**2") # collecting area
redshift = 0.05

So, that's everything--let's create the photons! We use the `make_photons` function for this:

In [ ]:
n_photons, n_cells = pyxsim.make_photons("sloshing_photons", sp, redshift, area, 
                                         exp_time, source_model)

Ok, that was easy. Now we have a photon list that we can use to create events using the `project_photons` function. Here, we'll just do a simple projection along the z-axis, and center the photons at RA, Dec = (45, 30) degrees. Since we want to be realistic, we'll want to apply foreground galactic absorption using the `"tbabs"` model, assuming a neutral hydrogen column of $N_H = 4 \times 10^{20}~{\rm cm}^{-2}$:

In [ ]:
n_events = pyxsim.project_photons("sloshing_photons", "sloshing_events", 
                                  "z", (45.,30.), absorb_model="tbabs", nH=0.04)

Now that we have a set of "events" on the sky, we can read them in and write them to a SIMPUT file:

In [ ]:
events = pyxsim.EventList("sloshing_events.h5")
events.write_to_simput("sloshing", overwrite=True)

We can then use this SIMPUT file as an input to the instrument simulator in SOXS. We'll use a small exposure time (100 ks instead of 300 ks), and observe it with the as-launched ACIS-I model:

In [ ]:
soxs.instrument_simulator("sloshing_simput.fits", "evt.fits", (100.0, "ks"), 
                          "chandra_acisi_cy0", [45., 30.], overwrite=True)

We can use the `write_image()` function in SOXS to bin the events into an image and write them to a file, restricting the energies between 0.5 and 2.0 keV:

In [ ]:
soxs.write_image("evt.fits", "img.fits", emin=0.5, emax=2.0, overwrite=True)

Now we can take a quick look at the image: 

In [ ]:
soxs.plot_image("img.fits", stretch='sqrt', cmap='arbre', vmin=0.0, vmax=10.0, width=0.2)